<a href="https://colab.research.google.com/github/DeepLearning-Y4/TrafficSignRecognition/blob/main/TrafficSignRecognition_ResNet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TrafficSignRecognition_ResNet_model



In [ ]:
# Mount Google Drive (optional if you stored the dataset there)
from google.colab import drive
drive.mount('/content/drive')